This program is part of pyHNC, copyright (c) 2023 Patrick B Warren (STFC).
Additional modifications copyright (c) 2025 Joshua F Robinson (STFC).
Email: patrick.warren{at}stfc.ac.uk.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see
<http://www.gnu.org/licenses/>.

Demonstrate the capabilities of the HNC package for solving DPD
potentials, comparing with SunlightHNC if requested, and plotting
the pair distribution function and the structure factor too.  For
details here see also the SunlightHNC documentation.

For standard DPD at $A = 25$ and $ρ = 3$, we have the following table

```
          ∆t = 0.02   ∆t = 0.01   Monte-Carlo  HNC   deviation
pressure  23.73±0.02  23.69±0.02  23.65±0.02   23.564  (0.4%)
energy    13.66±0.02  13.64±0.02  13.63±0.02   13.762  (1.0%)
mu^ex     12.14±0.02  12.16±0.02  12.25±0.10   12.170  (0.7%)
```


The first two columns are from dynamic simulations.  The excess
chemical potential (final row) is measured by Widom insertion.  The
HNC results from the present code are in agreement with those from
SunlightHNC to at least the indicated number of decimals.  The
deviation is between HNC and simulation results.

Data is from a forthcoming publication on osmotic pressure in DPD.

## Standard modules

In [1]:
import os
import pyHNC
import argparse
import pandas as pd
import numpy as np, matplotlib.pyplot as plt
from numpy import pi as π
from scipy.integrate import simpson
from pyHNC import Grid, PicardHNC, SolutePicardHNC, truncate_to_zero
import matplotlib.pyplot as plt

In [2]:
plt.rcParams['figure.figsize'] = 10, 7

## Global parameters

In [ ]:
N = 2**15
Δr = 0.01
grid = Grid(N, Δr)
r, q = grid.r, grid.q

verbose = False

alpha = 0.2
npicard = 1000
tol = 1e-12
solvent = PicardHNC(grid, alpha=alpha, npicard=npicard, tol=tol)
print(grid.details + '\n' + solvent.details)

In [ ]:
round(np.log2(grid.ng))

# 1. Solve for pure solvent

Define interaction parameters:

In [5]:
def dpd_potential(A, r):
    """Define the DPD potential."""
    return truncate_to_zero(A/2*(1-r)**2, r, 1)

def dpd_force(A, r):
    """Define dereivative of the DPD potential."""
    return truncate_to_zero(A*(1-r), r, 1) # the force f = -dφ/dr

# Parameters for solvent-solvent interactions
A00 = 25
ρ0 = 3.0
φ0, f0 = dpd_potential(A00, r), dpd_force(A00, r)

Solve for $h(r)$ via Picard iteration with HNC closure:

In [ ]:
soln = solvent.solve(φ0, ρ0, monitor=verbose) # solve for the DPD potential
h00, c00, h00q = soln.hr, soln.cr, soln.hq # extract for use in a moment

plt.plot(r, 1+h00)
plt.xlabel('$r$')
plt.ylabel('$g(r)$')
plt.xlim([0, 3])
plt.show()

In [ ]:
S00q = 1 + ρ0*h00q # solvent structure factor

plt.plot(q, S00q)
plt.xlabel('$q$')
plt.ylabel('$S(q)$')
plt.xlim([0, 25])
plt.show()

Calculate thermodynamic quantities using the total correlation function $h(r)$.

For the integrals here, see Eqs. (2.5.20) and (2.5.22) in Hansen & McDonald, "Theory of Simple Liquids" (3rd edition): for the (excess) energy density,
$$
e \equiv \frac{U^\mathrm{ex}}{V} = 2\pi\rho^2 \int_0^\infty \mathrm{d}r \, r^2 \phi(r) g(r)
$$
and virial pressure,
$$
p = \rho + \frac{2\pi\rho^2}{3} \int_0^\infty \mathrm{d}r \, r^3 f(r) g(r)
$$
where $f(r) = −\mathrm{d}\phi/\mathrm{d}r$ is the force. Note that we have assumed $\beta = 1$ in our expressions, so energy is given in units of $k_\mathrm{B} T$. An integration by parts shows that the mean-field contributions, being these with g(r) = 1, are the same.

Here specifically the mean-field contributions are
$$
\frac{2\pi \rho^3}{3} \int_0^\infty \mathrm{d}r \, r^3 f(r) = A \int_0^1 \mathrm{d}r \, r^3 (1−r) = \frac{\pi A \rho^3}{30} \,.
$$

In [ ]:
def excess_chemical_potential(h, c, r):
    return 4*π*ρ0 * simpson(r**2*(h*(h-c)/2 - c), r)

μ_ex = excess_chemical_potential(h00, c00, r)
μ = np.log(ρ0) + μ_ex
print(f'μ={μ:.4f} μ_ex={μ_ex:.4f} μ_id={μ-μ_ex:.4f}')

# 2. Introduce solute

We need to use a different solver that takes into account that this is now a binary mixture where the second species (the solute) is infinitely dilute. We need to pass the previously obtained static properties of the solvent.

In [9]:
solute = SolutePicardHNC(ρ0*h00q, grid, alpha=alpha, npicard=npicard, tol=tol)

Show how the distribution function $g_{01}$ between solvent (species 0) and solute (species 1) varies with DPD interaction parameter $A_{01}$. We can also infer $g_{11}$ directly from $g_{01}$ and thereby calculate the potential of mean force between two solute particles:
$$
- \ln{g_{11}(r)} = \beta \phi_{11}(r) + \beta \Delta \Omega_{11}(r)\,.
$$
Here $\beta \Delta \Omega_{11}$ is the depletion potential between solutes.

In [ ]:
fig1 = plt.figure(figsize=(3.375, 3))
ax1 = plt.gca()
fig2 = plt.figure(figsize=(3.375, 3))
ax2 = plt.gca()

ax1.plot(r, 1+h00, label=(r'$g_{00}(r; A_{00}=' + f'{A00})$'))

for A01 in np.arange(2*A00, 151, 25):
    φ01 = dpd_potential(A01, r)
    soln = solute.solve(φ01, monitor=verbose)
    h01, c01, h01q = soln.hr, soln.cr, soln.hq
    ax1.plot(r, 1+h01, label=(r'$g_{01}(r; A_{01}=' + f'{A01})$'))

    c01q = grid.fourier_bessel_forward(c01)
    depletion11 = - ρ0 * grid.fourier_bessel_backward(c01q * h01q)
    pl, = ax2.plot(r, depletion11, label=(r'$A_{01}=' + f'{A01}$'))

    # Equivalent calculation should be on top of other lines.
    # psi1q = h01q / (1 + ρ0*h00q)**0.5
    # depletion11 = - ρ0 * grid.fourier_bessel_backward(psi1q**2)
    # ax2.plot(r, depletion11, '--', c=pl.get_color())

for ax in [ax1, ax2]:
    ax.legend(loc='best', fontsize=8)
    ax.set_xlabel('$r$')
    ax.set_xlim([0, 3])

ax1.set_ylabel('$g(r)$')
ax2.set_ylabel(r'$\beta \Delta \Omega_{11}(r)$')

fig1.show()
fig2.show()

Calculate excess chemical potentials for solutes with varying solvent-solute interaction parameters $A_{01}$:

In [11]:
A01 = np.linspace(0, 60, 61)
μ_ex = np.empty(len(A01))
pmf_overlap = np.empty(len(A01))

for i, A in enumerate(np.flipud(A01)[:-1]):
    φ01 = dpd_potential(A, r)
    soln = solute.solve(φ01, monitor=verbose)
    h01, c01, c01q, h01q = soln.hr, soln.cr, soln.cq, soln.hq
    μ_ex[len(A01)-i-1] = excess_chemical_potential(h01, c01, r)
    pmf_overlap[len(A01)-i-1] = - ρ0 / (2*π**2) * simpson(q**2*c01q*h01q, q)

μ_ex[0] = 0.0
pmf_overlap[0] = 0.0

In [ ]:
np.flipud(np.linspace(0, 60, 13))

Plot the previously calculated chemical potentials:
digitised from Fig 1 of Hendrikse *et al.*, PCCP **27**, 1554-66 (2025).


In [ ]:
schema2 = {'quantity':str, 'A01':float, 'value':float, 'error':float, 'njobs':int, 'file':str}
muref = pd.read_csv('muref_all.dat', sep='\t', names=schema2.keys(), dtype=schema2)
muref

In [ ]:
A01 = 35
slice = (muref['quantity']=='mu') & (muref['A01'] == A01)
muref[slice]

In [ ]:
muref['njobs'].unique()

In [ ]:
slice = (muref['quantity']=='mu') & (muref['file'] == 'muref2_mu.dat')
muex = muref[slice].set_index('A01')['value']
muex_err = muref[slice].set_index('A01')['error']
#muex = muref[slice][['A01', 'value']].groupby('A01').mean()['value']
pd.concat([muex, muex_err], axis=1)

In [ ]:
slice = (muref['quantity']=='mu') & muref['file'].str.startswith('muref2')
muex = muref[slice][['A01', 'value']].groupby('A01').mean()['value']
muex

In [ ]:
schema = {'quantity':str, 'A01':float, 'dlength':float, 'value':float, 'error':float, 'njobs':int, 'file':str}
dimer = pd.read_csv('dimer1_all.dat', sep='\t', names=schema.keys(), dtype=schema)
dimer

In [ ]:
schema = {'quantity':str, 'A01':float, 'dlength':float, 'value':float, 'error':float, 'njobs':int, 'file':str}
dimers = pd.read_csv('dimer2_all.dat', sep='\t', names=schema.keys(), dtype=schema)
dimers

In [ ]:
np.sort(dimer['A01'].unique()), np.sort(dimers['A01'].unique())


In [ ]:

plt.figure(figsize=(3.375, 3.375))

plt.plot([0, 2], [0, 0], 'k:')
for A01 in np.linspace(5, 25, 5):
    slice = (dimer['quantity'] == 'mu') & (dimer['A01'] == A01)
    muexdim = dimer[slice].groupby('dlength').value.mean()
    plt.plot(muexdim.index, (muexdim-2*muex[A01])/(2*muex[A01]-muex[2*A01]), 'o-', label=f'A01 = {A01}')
    
d = np.linspace(0, 2, 41)
g = (1+2*d)*(1-d)**2
g[d>1] = 0
plt.plot(d, -g, 'k--', label='$(1+2d)(1-d)^2\;[d < 1]$')
R = 1
vov = (1+d/(4*R))*(1-d/(2*R))**2
vov[d>2*R] = 0
plt.plot(d, -vov, 'k-.', label='$(1+d/4)(1-d/2)^2\;[d<2]$')
plt.legend()
plt.xlabel('d')
plt.ylabel('reduced PMF')

In [ ]:
slice = (dimers['quantity'] == 'mu') & (dimers['A01'] == 35)
ser35 = dimers[slice][['dlength', 'value']].groupby('dlength').mean()['value']
ser35

In [ ]:
slice = (mudimer['quantity'] == 'mu') & (mudimer['dlength'] == 0.0)
mudimer[slice][['A01', 'value']].groupby('A01').mean()['value']

In [ ]:
slice = (muref['quantity']=='mu') & (muref['A01'] % 10 == 0)
muref[slice].set_index('A01')

In [ ]:
slice = (muref['quantity']=='mu') & muref['file'].str.startswith('muref2') & (muref['A01'] % 10 == 0)
muref[slice].set_index('A01')['value']

In [ ]:
# plt.plot(A01, μ_ex, '-', label='HNC')

# μ_rpa = ρ0 * np.pi * A01 / 15
# plt.plot(A01, μ_rpa, '--', label='RPA')

slice = (muref['quantity']=='mu') & muref['file'].str.startswith('muref2')
ser = muref[slice][['A01', 'value']].groupby('A01').value.mean()
plt.plot(ser.index, ser, 'o', label='DPD')

plt.xlabel(r'$A_{01}$')
plt.ylabel(r'$\mu_1^\mathrm{ex}$')
plt.legend(loc='upper left')
plt.xlim([0, np.max(A01)])
plt.ylim([0, 25])

In [ ]:
A01 = np.linspace(0, 60, 13)
μ_ex = np.empty(len(A01))
pmf_overlap = np.empty(len(A01))

for i, A in enumerate(np.flipud(A01)[:-1]):
    φ01 = dpd_potential(A, r)
    soln = solute.solve(φ01, monitor=verbose)
    h01, c01, c01q, h01q = soln.hr, soln.cr, soln.cq, soln.hq
    μ_ex[len(A01)-i-1] = excess_chemical_potential(h01, c01, r)
    pmf_overlap[len(A01)-i-1] = - ρ0 / (2*π**2) * simpson(q**2*c01q*h01q, q)

μ_ex[0] = 0.0
pmf_overlap[0] = 0.0

df = pd.DataFrame(np.array([A01, pmf_overlap, μ_ex]).transpose(), columns=['A01', 'PMF0', 'mu_ex'])
df['PMF0+2muex'] = df['PMF0'] + 2*df['mu_ex']
df

In [ ]:
plt.plot(A01, pmf_overlap+2*μ_ex, '-', label='PMF')
plt.plot(A01, μ_ex, '-', label='mu')

plt.xlabel(r'$A_{01}$')
plt.ylabel(r'$\beta \Delta \Omega_{11}(0)$')
plt.legend(loc='upper left')
plt.xlim([0, np.max(A01)])
#plt.ylim([0, 25])


In [ ]:
muex = muref[muref['file'].str.startswith('muref2') & (muref['quantity'] == 'mu')].set_index('A01')['value']
muex

In [22]:
saathoff_15 = np.loadtxt('Saathoff_JCP2018_fig5a_A12equals15.dat').transpose()
saathoff_25 = np.loadtxt('Saathoff_JCP2018_fig5a_A12equals25.dat').transpose()
saathoff_35 = np.loadtxt('Saathoff_JCP2018_fig5a_A12equals35.dat').transpose()


In [ ]:
plt.plot(saathoff_15[0], saathoff_15[1])

In [ ]:
A01 = 50
slice = (dimer['quantity'] == 'mu') & (dimer['A01'] == A01)


In [13]:
dimer_all = pd.concat([dimer, dimers], ignore_index=True, sort=False)

In [ ]:
A02 = 35
slice = (dimer_all['quantity'] == 'mu') & (dimer_all['A01'] == A01)
dimer_all[slice][['dlength', 'value']].groupby('dlength').mean()['value']


In [ ]:
np.arange(0, 35, 5)

In [ ]:
df2 = pd.DataFrame([(A01, muref_ser[2*A01]) for A01 in df.index if 2*A01 in muref_ser],
                   columns=['A01', 'muref(2*A01)']).set_index('A01')
df2

In [ ]:
slice = (dimer_all['quantity'] == 'mu') & (dimer_all['dlength'] == 0)
df1 = dimer_all[slice][['A01', 'value']].groupby('A01').mean()
df1.rename(columns={'value': 'PMF(0)'}).join(df2)

In [ ]:
df1.join(df2)

In [ ]:
for A01 in np.arange(0, 40, 5):
    φ01 = dpd_potential(A01, r)
    soln = solute.solve(φ01, monitor=verbose)
    h01, c01, h01q = soln.hr, soln.cr, soln.hq
    c01q = grid.fourier_bessel_forward(c01)
    depletion11 = - ρ0 * grid.fourier_bessel_backward(c01q * h01q)
    plt.plot(r, depletion11, label=(r'$A_{01}=' + f'{A01}$'))
    c = plt.gca().lines[-1].get_color()
    slice = (dimer_all['quantity'] == 'mu') & (dimer_all['A01'] == A01)
    ser = dimer_all[slice][['dlength', 'value']].groupby('dlength').mean()['value']
    plt.plot(ser.index, ser-2*muref_ser[A01], 'o', color=c)

#plt.plot(saathoff_15[0], saathoff_15[1], 'kx--', label='Saathoff 15')
#plt.plot(saathoff_25[0], saathoff_25[1], 'rx--', label='Saathoff 25')
#plt.plot(saathoff_35[0], saathoff_35[1], 'bx--', label='Saathoff 35')

    
plt.legend(loc='best', fontsize=8)
plt.xlabel('$r$')
plt.xlim([0, 2])

plt.ylabel(r'$\beta \Delta \Omega_{11}(r)$')


In [ ]:
slice = (muref['quantity'] == 'um')
df1 = muref[slice][['A01', 'value']].groupby('A01').mean()*2
df1.rename(columns={'value': '2<U>'}, inplace=True)
df1

In [ ]:
slice = (dimers['quantity'] == 'um')
df2 = dimers[slice][['A01', 'value']].groupby('A01').mean()
df2.rename(columns={'value': '<U1+U2>'}, inplace=True)
df2

In [ ]:
df1.join(df2)

In [ ]:
- ρ0 / (2*π**2) * simpson(q**2*c01q*h01q, q)

In [ ]:
df7[df7['A01']==25]

# 3. Partition coefficients of dimers

Taking the depletion potential from the previous section we can integrate over all configurations of a dimer to obtain its chemical potential:

In [ ]:
def bond_potential(r, l0=0.5, k=150):
    """Spring force between bonded atoms in a molecule."""
    return k * (r - l0)**2

from scipy.integrate import simpson

def dimer_excess_chemical_potential(A1, A2, l0=0.5, k=150):
    """Excess chemical potential of a non-rigid dimer is found by integrating
    the potential of mean force over all configurations of the two beads.

    Args:
        A1: interaction strength of bead 1 with the solvent.
        A2: interaction strength of bead 2 with the solvent.
        l0: equilbrium length of dimer in a vacuum.
        k: interaction strength between the two beads (a spring constant).
    Returns:
        Excess chemical potential in units of kT.
    """
    φ01, φ02 = [dpd_potential(A, r) for A in [A1, A2]]
    soln = solute.solve(φ01, monitor=verbose)
    h01, c01, h01q = soln.hr, soln.cr, soln.hq
    soln = solute.solve(φ02, monitor=verbose)
    h02, c02, h02q = soln.hr, soln.cr, soln.hq

    # Depletion contribution to potential of mean force
    psi1q, psi2q = [hq / (1 + ρ0*h00q)**0.5 for hq in [h01q, h02q]]
    depletion12 = - ρ0 * grid.fourier_bessel_backward(psi1q*psi2q)

    v12 = bond_potential(r, l0, k)
    φ12 = v12 + depletion12
    return np.log(simpson(4*np.pi * r**2 * np.exp(-φ12), r))

A1 = 25
A2 = np.arange(5, 76, 5)
μ_ex_dimer = [dimer_excess_chemical_potential(A1, A) for A in A2]
plt.plot(A2, μ_ex_dimer)
plt.xlabel('$A_{02}$')
plt.ylabel(r'$\beta \mu_\text{dimer}^\text{ex}$')
plt.title(r'$A_{01} = 25$')
plt.show()


The partition coefficient is obtained from the ratio of excess chemical potentials in two solvents $\alpha$ and $\beta$:
$$
\ln{K_{ij}^{\alpha\beta}}
\equiv
\ln{\left( \frac{c_{ij}^\alpha}{c_{ij}^\beta} \right)}
=
\beta \mu_{ij,\alpha}^\text{ex} - \beta \mu_{ij,\beta}^\text{ex}\,.
$$

In [ ]:
def monomer_partition_coefficient(Aα1, Aβ1):
    """Partition coefficient of a monomer for two reference solvents.
    Args:
        Aα1: interaction coefficient between solute and first solvent.
        Aβ1: interaction coefficient between solute and second solvent.
    Returns:
        Difference between excess chemical potentials in units of kT.
    """
    φα1, φβ1 = [dpd_potential(A, r) for A in [Aα1, Aβ1]]

    soln = solute.solve(φα1, monitor=verbose)
    hα1, cα1, hα1q = soln.hr, soln.cr, soln.hq
    μ_ex_α = excess_chemical_potential(hα1, cα1, r)

    soln = solute.solve(φβ1, monitor=verbose)
    hβ1, cβ1, hβ1q = soln.hr, soln.cr, soln.hq
    μ_ex_β = excess_chemical_potential(hβ1, cβ1, r)

    return μ_ex_β - μ_ex_α

def dimer_partition_coefficient(A1, A2, l0=0.5, k=150):
    """Partition coefficient of a dimer for two reference solvents.
    Args:
        A1: interaction coefficients of first bead with each solvent.
        A2: interaction coefficients of second bead with each solvent.
        l0: equilbrium length of dimer in a vacuum.
        k: interaction strength between the two beads (a spring constant).
    Returns:
        Difference between excess chemical potentials in units of kT.
    """

    K1 = monomer_partition_coefficient(*A1)
    K2 = monomer_partition_coefficient(*A2)

    Aα1, Aβ1 = A1
    Aα2, Aβ2 = A2
    μ_ex_α = dimer_excess_chemical_potential(Aα1, Aα2, l0, k)
    μ_ex_β = dimer_excess_chemical_potential(Aβ1, Aβ2, l0, k)

    return K1 + K2 + μ_ex_α - μ_ex_β

A1 = [25, 25]
A2 = [100, 25]
K1 = monomer_partition_coefficient(*A1)
K2 = monomer_partition_coefficient(*A2)

l0 = np.linspace(0, 3, 100)
logP = [dimer_partition_coefficient(A1, A2, ll) for ll in l0]

plt.plot(l0, logP)
plt.axhline(y=(K1+K2), ls='--', lw=0.5)
plt.xlabel('$l_0$')
plt.ylabel(r'$\ln{K_{12}}$')
plt.title(r'$A_{\alpha 1} = 25, A_{\alpha 2} = 100, A_{\beta 1} = A_{\beta 2} = 25$')
plt.show()